# Exercício 1 - Enunciado


1. O algoritmo estendido de Euclides (EXA) aceita dois inteiros constantes  $\,a,b>0\,$  e devolve inteiros $r,s,t\,$ tais que  $\,a*s + b*t = r\,$  e  $\,r = \gcd(a,b)\,$. 
    Para além das variáveis $\,r,s,t\,$ o código requer 3 variáveis adicionais $\,r',s',t'\,$ que representam os valores de $\,r,s,t\,$ no “próximo estado”.
    ```
    INPUT  a, b
    assume  a > 0 and b > 0
    r, r', s, s', t, t' = a, b, 1, 0, 0, 1
    while r' != 0
      q = r div r'
      r, r', s, s', t, t' = r', r − q × r', s', s − q × s', r', t − q × t' 
    OUTPUT r, s, t
    ````
    1. Construa um FOTS usando BitVector de tamanho $n$ que descreva o comportamento deste programa: identifique as variáveis do modelo, o estado inicial e a relação de transição.
    2. Considere estado de erro quando $\,r=0\,$ ou alguma das variáveis atinge o “overflow”.
        Prove que o programa nunca atinge o estado de erro
    3. Prove que a relação de Bézout $\,$ $\,a*s + b*t = r\,$ é um invariante do algoritmo.

# Exercício 1 - Solução

s é o coeficiente pelo qual 'a' multiplica
t é o coeficiente pelo qual 'b' multiplica
r é o máximo divisor comum de 'a' e 'b'

a, b > 0'\n'  
a\*s + b\*t = r = mdc(a, b)  
r = mdc(a,b)

r', s', t' representam r, s, t no próximo estado

1. Fazer função declare
2. Fazer função init
3. Fazer função trans
4.  r = 0 ou s,t overflow é estado de erro. Provar que nunca atinge estado de erro
5. Provar que a*s + b*t = r é um invariante

In [39]:
from pysmt.shortcuts import *
from pysmt.typing import INT

In [40]:
def declare(i):
    state = {}
    state['s'] = Symbol('s'+str(i),INT)
    state['t'] = Symbol('t'+str(i),INT)
    state['r'] = Symbol('r'+str(i),INT)
    state['s_prox'] = Symbol('s_prox'+str(i),INT)
    state['t_prox'] = Symbol('t_prox'+str(i),INT)
    state['r_prox'] = Symbol('r_prox'+str(i),INT)
    state['q'] = Symbol('q'+str(i),INT)

    return state

In [41]:
def init(state, a, b):
    a = int(a)
    b = int(b)
    
    A = Equals(state['r'], Int(a))
    B = Equals(state['r_prox'], Int(b))
    C = Equals(state['s'], Int(1))
    D = Equals(state['s_prox'], Int(0))
    E = Equals(state['t'], Int(0))
    F = Equals(state['t_prox'], Int(1))
    
    return And(A, B, C, D, E, F)

In [42]:
def trans(curr):
    t = And(Equals(curr['q'], Div(curr['r'], curr['r_prox'])),
            Equals(curr['r'], curr['r_prox']), 
            Equals(curr['r_prox'], Times(Minus(curr['r'], curr['q']), curr['r_prox'])), 
            Equals(curr['s'], curr['s_prox']), 
            Equals(curr['s_prox'], Minus(curr['s'], Times(curr['q'], curr['s_prox']))),
            Equals(curr['t'], curr['t_prox']),
            Equals(curr['t_prox'], Minus(curr['t'], Times(curr['q'], curr['t_prox']))),
            )
    return t

In [51]:
def combine(i, a, b):
    states = declare(i)
    
    formula = And(Or(Not(Equals(states['r'], Int(0))), trans(states)))

    with Solver() as solver:
        # Falta algum while para verificar a condição do ciclo (pelo menos a meu ver)
        solver.add_assertion(init(states, a, b))
        solver.add_assertion(formula)
        
        #solver.push()
        solver.add_assertion(Equals(Plus(Times(Int(a), states['s']), Times(Int(b), states['t'])), states['r']))
        
        if solver.solve():
            for key, value in states.items():
                print(f"> State {key}: {key} = {solver.get_value(value)}")
        else:
            print("> Not feasible.")
        
        #solve.pop()

combine(0, 5, 6)

> State s: s = 1
> State t: t = 0
> State r: r = 5
> State s_prox: s_prox = 0
> State t_prox: t_prox = 1
> State r_prox: r_prox = 6
> State q: q = 0
